In [ ]:
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
# For reproductibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
# parameters
batch_size = 10

# MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
mnist_train = datasets.MNIST(root="../MNIST_data/",train=True,
                          transform=transform, download=False)
mnist_test  = datasets.MNIST(root="../MNIST_data/",train=False,
                          transform=transform, download=False)

# Dataset Loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
W1 = torch.nn.Parameter(torch.Tensor(784,30)).to(device)
b1 = torch.nn.Parameter(torch.Tensor(30)).to(device)
W2 = torch.nn.Parameter(torch.Tensor(30,10)).to(device)
b2 = torch.nn.Parameter(torch.Tensor(10)).to(device)

torch.nn.init.normal_(W1)
torch.nn.init.normal_(b1)
torch.nn.init.normal_(W2)
torch.nn.init.normal_(b2)

In [ ]:
def sigmoid(x):
    return 1.0 / (1.0 + torch.exp(-x))
def dsigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [ ]:
X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)[:1000]
Y_test = mnist_test.targets.to(device)[:1000]
X_test.size()
Y_test.size()

In [ ]:
lr = 0.5
for i in range(10000):
    for X, Y in data_loader:
        # forward
        X = X.view(-1, 28 * 28).to(device)
        Y = torch.zeros((batch_size, 10)).scatter_(1, Y.unsqueeze(1), 1).to(device)
        # One-hot encoding
        l1 = torch.add(torch.matmul(X ,W1),b1)
        a1 = sigmoid(l1)
        l2 = torch.add(torch.matmul(a1,W2),b2)
        Ey = sigmoid(l2)
        
        
        diff = Ey - Y
        
        # Backprop
        dl2 = diff * dsigmoid(l2)
        db2 = dl2
        dW2 = torch.matmul(torch.transpose(a1,0,1), dl2)
        
        da1 = torch.matmul(dl2, torch.transpose(W2,0,1))
        
        dl1 = da1 * dsigmoid(l1)
        db1 = dl1
        dW1 = torch.matmul(torch.transpose(X,0,1), dl1)
        
        W1 = W1 - lr * dW1
        b1 = b1 - lr * torch.mean(db1,0)
        W2 = W2 - lr * dW2
        b2 = b2 - lr * torch.mean(db2,0)
        idx = i;
    if idx % 1000 == 0:
        # One-hot encoding
        l1 = torch.add(torch.matmul(X_test,W1),b1)
        a1 = sigmoid(l1)
        l2 = torch.add(torch.matmul(a1,W2),b2)
        Ey = sigmoid(l2)
        accuracy_mat = torch.argmax(Ey,1) == Y_test
        accuracy_res = accuracy_mat.sum()
        print("{:5d} Accuracy : {}/{}".format(i, accuracy_res,Y_test.size()))
    del X
    del Y

In [ ]:

# model
# model = torch.nn.Sequential(linear1,sigmoid,linear2,sigmoid,
#                             linear3,sigmoid,linear4,sigmoid).to(device)